# 1  数值稳定性
- 神经网络变深以后出现问题
- 梯度计算利用链式法则
- 假设有d层，目前在t层，每层梯度是一个矩阵，d-t个矩阵乘法 ，溢出问题
    - 梯度爆炸：大于1
    - 梯度消失：小于1
    - MLP：不考虑偏置，$h_t=\sigma(W_th_{t-1})$ 向量对向量求导
    - 对角矩阵×$W_t^T$:因为$\sigma$是作用在元素上的，所以非对角梯度=0


##  ReLU激活函数
- $\sigma(x)= max\{0,x\}$
- $\prod_{i=1}^{d-t}链式法则=\prod_{i=1}^{d-t}diag{\sigma'(W_th_{t-1})W_t^T}$
- 导数部分为1或者0，所以梯度只保留了W的某些行的累积
- 如果层数很深，那么梯度将会很大
- 值可能超出值域inf 6e-5 6e4
- 影响：
  - 学习率大，大参数值，更大的梯度，死循环
  - 学习率小，小参数，更小的梯度，训练无进展
  - 需要不断调整学习率

##  sigmoid 激活函数
- 梯度是 本身×（1-本事）
- 中间值梯度最大，两边梯度很小，输入稍微大一点，梯度就变得很小
- d-t个小数值得乘积
- 影响：
  - 梯度变为0，对16位浮点数尤为严重
  - 训练不会有进展，不论lr是多少
  - 对底部层影响严重
  - 顶部训练好，无法让神经网络更深

# 2 让训练更加稳定
- 让梯度值在合理的范围内： 1e-6,1e3
- 将累乘变加法 :ResNet LSTM
- 归一化：梯度归一化，梯度裁剪
   - 合理的初始权重和激活函数

## 每层的方差保持常数
- 让每一层输出和梯度看作随机变量
- 每一层均值方差保持一致：一般均值为0，方差为a
- 反向梯度和每层输出假设一致:均值为0，方差为b

## 权重初始化
- 合理的区间内随机初始参数：训练开始更容易产生数值不稳定
  - 远离最优解：损失函数表面较复杂
  - 靠近最优解:  损失函数表明平滑
  - 正态分布可能小网络没问题，更深的网络可能出现问题
  - 激活函数的非线性改变了期望和方差 
     - 正向需要保证$n_{t-1}\gamma_t=1$才能方差不变 $n_{t-1}$为神经元个数
     - 反向需要保证$n_t\gamma_t=1$保证梯度方差不变
     - 除非输入和输出个数相同，否则无法保证满足两个条件
  - Xavier初始权重$(n_{t-1}+n_t)\gamma_t/2=1$
  - 设置初始参数分布的方差为 $\gamma_t=2/(n_{t-1}+n_t)$ 

## 线性激活函数
- 等于本身才能保证方差不变
- 将常见的激活函数泰勒展开: 0点附近f(x)=x
  - sigmoid不满足，调整为4sigmoid-2
  - tanh满足
  - ReLU满足